In [ ]:
# %pip install pandas
# %pip install psycopg2
# %pip install sqlalchemy

In [3]:
import pandas as pd
from sqlalchemy import create_engine

In [1]:
class ProcessandoDados:
    def __init__(self, db_config):
        self.db_config = db_config
        self.engine = create_engine(
            f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
        )

    def carregar_dados(self, file_path, delimiter=';'):
        """Carrega os dados de um arquivo CSV."""
        pd.set_option('display.max_columns', None)
        df = pd.read_csv(file_path, delimiter=delimiter)
        return df

    def analisar_dados(self, df):
        """Exibe informações sobre os dados, como nulos, duplicados, únicos e tipos de colunas."""
        print("Valores nulos por coluna:\n", df.isnull().sum())
        print("\nQuantidade de valores duplicados:\n", df.duplicated().sum())
        print("\nQuantidade de valores únicos por coluna:\n", df.nunique())
        print("\nTipos de dados por coluna:\n", df.dtypes)

    def process_columns(self, df, year):
        """Filtra colunas relevantes para um determinado ano."""
        cols = df.columns[df.columns.str.contains("NOME") | df.columns.str.contains(str(year))]
        df_filtered = df[cols].dropna(subset=[f"INDE_{year}"])
        df_filtered.columns = df_filtered.columns.str.replace(f"_{year}", "").str.lower()
        df_filtered['ano'] = year
        return df_filtered

    def process_year(self, df, year):
        """Processa os dados para um ano específico e retorna o dataframe correspondente."""
        return self.process_columns(df, year)

    def salvar_no_rds(self, df, table_name):
        """Salva o dataframe em uma tabela no banco de dados RDS."""
        df.to_sql(table_name, con=self.engine, if_exists='replace', index=False)
        print(f"Dados salvos na tabela {table_name} com sucesso.")

In [4]:
config = {
    "host": "seu-host-rds.amazonaws.com",
    "port": "5432",
    "user": "seu-usuario",
    "password": "sua-senha",
    "database": "seu-banco"
}

df = None
processador = None

if __name__ == "__main__":
    processador = ProcessandoDados(config)
    file_path = "./source/PEDE_PASSOS_DATASET_FIAP.csv"

    df = processador.carregar_dados(file_path)
    # processador.analisar_dados(df)

    for year in [2020, 2021, 2022]:
        processed_df = processador.process_year(df, year)
        table_name = f"dados_{year}"
        # processador.salvar_no_rds(processed_df, table_name)
        print(f"\nDados processados para o ano {year} e salvos na tabela {table_name}")


Dados processados para o ano 2020 e salvos na tabela dados_2020

Dados processados para o ano 2021 e salvos na tabela dados_2021

Dados processados para o ano 2022 e salvos na tabela dados_2022


In [7]:
processador.analisar_dados(df)

Valores nulos por coluna:
 INSTITUICAO_ENSINO_ALUNO_2020    621
NOME                               0
IDADE_ALUNO_2020                 621
ANOS_PM_2020                     621
FASE_TURMA_2020                  621
                                ... 
INDICADO_BOLSA_2022              487
PONTO_VIRADA_2022                487
IPV_2022                         487
IAN_2022                         487
NIVEL_IDEAL_2022                 487
Length: 69, dtype: int64

Quantidade de valores duplicados:
 0

Quantidade de valores únicos por coluna:
 INSTITUICAO_ENSINO_ALUNO_2020       8
NOME                             1349
IDADE_ALUNO_2020                   15
ANOS_PM_2020                        6
FASE_TURMA_2020                    63
                                 ... 
INDICADO_BOLSA_2022                 2
PONTO_VIRADA_2022                   2
IPV_2022                          452
IAN_2022                            3
NIVEL_IDEAL_2022                    9
Length: 69, dtype: int64

Tipos de dados p

In [8]:
df_2020 =processador.process_year(df, 2020)
df_2021 =processador.process_year(df, 2021)
df_2022 =processador.process_year(df, 2022)

In [9]:
df_2020.iloc[0]

instituicao_ensino_aluno                                       Escola Pública
nome                                                                  ALUNO-1
idade_aluno                                                                11
anos_pm                                                                     2
fase_turma                                                                 2H
ponto_virada                                                              Não
inde                                                                 7.883752
inde_conceito                                                               B
pedra                                                                Ametista
destaque_ieg                Seu destaque em 2020: A sua boa entrega das li...
destaque_ida                Ponto a melhorar em 2021: Empenhar-se mais nas...
destaque_ipv                Seu destaque em 2020: A sua boa integração aos...
iaa                                                         8.50